$\Gamma_{E} = \{ \mathscr{X}, \mathscr{A}, p(\cdot), \alpha(\cdot), \mathscr{H}, H(\cdot), \iota(\cdot), \rho(\cdot), u \}$

In [ ]:
from itertools import product

class GameStructure:
    # 기본 입력: 노드 집합(X), 액션 집합(A), 경기자 집합(I), 인접행렬(ADJ), information set 모음(H), information set을 각 경기자에게 할당하는 함수(iota), nature 존재여부(nature)
    # 추가 입력: 터미널노드-보수 딕셔너리(payoffs), nature의 움직임에 대한 확률(rho)
    def __init__(self, X, A, I, ADJ, H, iota, nature, payoffs=None, rho=None):
        self.X = X
        self.A = A
        self.I = I
        self.H = H
        self.iota = iota
        self.nature = nature
        self.rho = rho
        self.payoffs = payoffs if payoffs else {}

        # 게임 구조 정의
        self.P, self.alpha = zip(*[(i, ADJ[i][j]) for i in X for j in X if i < j and ADJ[i][j] is not None])
        self.P, self.alpha = [None] + list(self.P), [None] + list(self.alpha)
        self.S = [([idx for idx, value in enumerate(self.P) if value == x] or None) for x in X]
        self.T = [idx for idx, value in enumerate(self.S) if value is None]
        self.C_X = [[value for idx, value in enumerate(self.alpha) if idx in s] for s in self.S if s is not None]
        self.C_H = [self.C_X[idx] for idx, value in enumerate(H) if value is not None and H.index(value) == idx]
        self.S_i = [[''.join(s) for s in product(*[self.C_H[idx] for idx, value in enumerate(iota) if value == i])] for i in I]


    # 게임 구조 출력 함수
    def display(self):
        print(f'nodes: X = {self.X}')
        print(f'actions: A = {self.A}')
        print(f'players: I = {self.I}')
        print(f'predecessors (X->X): P = {self.P}')
        print(f'successors: S = {self.S}')
        print(f'actions (X->A): alpha = {self.alpha}')
        print(f'terminal nodes: T = {self.T}')
        print(f'choices available at decision node: C(X) = {self.C_X}')
        print(f'information sets (X->H): H = {self.H}')
        print(f'choices available at information set: C(H) = {self.C_H}')
        print(f'players (H->I): iota = {self.iota}')

        if self.nature:
            print(f"player {0} is nature")
            print(f'probabilities(moves by nature) rho = {self.rho}')
        
        for i in self.I:
            print(f"player {i}'s strategy: S{i} = {self.S_i[i]}")


    def display_payoff_matrix(self):
        if not self.payoffs:
            print("no payoffs defined for this game.")

        # 경기자 목록 (nature 제외)
        human_players = [i for i in self.I if i != 0 or not self.nature]
        player_strategies = [self.S_i[i] for i in human_players]

        print("\npayoff matrix:")
        print(f"player {str(human_players[-1])} strategies: {'':^40}")
        print(" " * 5 + " " + " ".join([f"{strat:^8}" for strat in player_strategies[-1]]))

        # 터미널 노드에서 보수를 확인할 수 있으므로, 보수 행렬을 출력하기 위해서는 각 전략에서 연결되는 터미널 노드를 알아야함
        # 각 경기자의 전략 조합을 확인하고 보수 조회
        for strat1 in player_strategies[0]:
            row = f"player {human_players[0]} {strat1:<8}"

            for strat2 in player_strategies[-1]:
                current_node = self.X[0]  # 이니셜 노드에서 시작
                path = strat1 + strat2  # 두 경기자의 전략을 조합하여 이동 경로 설정
                action_idx = 0
                visited = set()  # 이미 거쳐간 노드 저장 

                while current_node not in self.T and action_idx < len(path) and current_node not in visited:
                    visited.add(current_node)
                    found = False
                    # 현재 노드에서 가능한 다음 노드 탐색
                    for idx in range(1, len(self.P)):
                        if self.P[idx]==current_node and self.alpha[idx] == path[action_idx]:
                            next_node_idx = self.X.index(current_node) + (idx - self.X.index(current_node))
                            current_node = self.X[next_node_idx]
                            action_idx += 1
                            found = True
                            break
                    if not found:
                        break  # 이동할 노드가 없으면 종료료

                # 터미널 노드에 도착하면 보수 출력 
                payoff = self.payoffs.get(current_node, (0, 0))
                row += f" {payoff} "
            print(row)



    def display_expected_payoff_matrix(self):
        if not self.nature or not self.rho:
            print("This method is only applicable for games with nature and rho defined.")
            return
        if not self.payoffs:
            print("No payoffs defined for this game.")
            return

        # 경기자 목록 (nature 제외)
        human_players = [i for i in self.I if i != 0]
        player_strategies = [self.S_i[i] for i in human_players]

        print("\nexpected payoff matrix (considering nature's moves):")
        print(f"player {str(human_players[-1])} strategies: {'':^40}")
        print(" " * 5 + " " + " ".join([f"{strat:^8}" for strat in player_strategies[-1]]))

        # 각 경기자의 전략 조합에 대해 기대 보수 계산
        for strat1 in player_strategies[0]:
            row = f"player {human_players[0]} {strat1:<8}"
            for strat2 in player_strategies[1]:
                expected_payoff = [0.0, 0.0]  # 경기자 1과 경기자 2의 기대 보수 초기화 
                nature_moves = [(1, self.S_i[0][0], self.rho[0]), (2, self.S_i[0][1], self.rho[1])]  # nature의 선택: (다음 노드, 액션, 확률)

                for next_node, _, prob in nature_moves:
                    current_node = next_node
                    path = strat1 + strat2  # 두 플레이어의 전략 조합
                    action_idx = 0
                    visited = set()

                    # 터미널 노드까지 경로 탐색
                    while current_node not in self.T and action_idx < len(path) and current_node not in visited:
                        visited.add(current_node)
                        found = False
                        for idx in range(1, len(self.P)):
                            if self.P[idx] == current_node and self.alpha[idx] == path[action_idx]:
                                next_node_idx = self.X.index(current_node) + (idx - self.X.index(current_node))
                                current_node = self.X[next_node_idx]
                                action_idx += 1
                                found = True
                                break
                        if not found:
                            break

                    # 터미널 노드에 도착하면 기대 보수 계산 
                    if current_node in self.T:
                        payoff = self.payoffs.get(current_node, (0, 0))
                        expected_payoff[0] += prob * payoff[0]
                        expected_payoff[1] += prob * payoff[1]

                row += f" ({expected_payoff[0]:.2f}, {expected_payoff[1]:.2f}) "
            print(row)

In [5]:
X = list(range(15))
A = ['a', 'b', 'c', 'd', 'e', 'f']
ADJ = [
    [None, 'a', 'b', None, None, None, None, None, None, None, None, None, None, None, None],
    ['a', None, None, 'c', 'd', None, None, None, None, None, None, None, None, None, None],
    ['b', None, None, None, None, 'e', 'f', None, None, None, None, None, None, None, None],
    [None, 'c', None, None, None, None, None, 'a', 'b', None, None, None, None, None, None],
    [None, 'd', None, None, None, None, None, None, None, 'a', 'b', None, None, None, None],
    [None, None, 'e', None, None, None, None, None, None, None, None, 'a', 'b', None, None],
    [None, None, 'f', None, None, None, None, None, None, None, None, None, None, 'a', 'b'],
    [None, None, None, 'a', None, None, None, None, None, None, None, None, None, None, None],
    [None, None, None, 'b', None, None, None, None, None, None, None, None, None, None, None],
    [None, None, None, None, 'a', None, None, None, None, None, None, None, None, None, None],
    [None, None, None, None, 'b', None, None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, 'a', None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, 'b', None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, 'a', None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, 'b', None, None, None, None, None, None, None, None]
]
I = [0, 1, 2]
H = [0, 1, 2, 3, 3, 4, 4, None, None, None, None, None, None, None, None]
iota = [0, 1, 1, 2, 2]

game_example = GameStructure(X, A, I, ADJ, H, iota, nature=False)
game_example.display()

nodes: X = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
actions: A = ['a', 'b', 'c', 'd', 'e', 'f']
players: I = [0, 1, 2]
predecessors (X->X): P = [None, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6]
successors: S = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], None, None, None, None, None, None, None, None]
actions (X->A): alpha = [None, 'a', 'b', 'c', 'd', 'e', 'f', 'a', 'b', 'a', 'b', 'a', 'b', 'a', 'b']
terminal nodes: T = [7, 8, 9, 10, 11, 12, 13, 14]
choices available at decision node: C(X) = [['a', 'b'], ['c', 'd'], ['e', 'f'], ['a', 'b'], ['a', 'b'], ['a', 'b'], ['a', 'b']]
information sets (X->H): H = [0, 1, 2, 3, 3, 4, 4, None, None, None, None, None, None, None, None]
choices available at information set: C(H) = [['a', 'b'], ['c', 'd'], ['e', 'f'], ['a', 'b'], ['a', 'b']]
players (H->I): iota = [0, 1, 1, 2, 2]
player 0's strategy: S0 = ['a', 'b']
player 1's strategy: S1 = ['ce', 'cf', 'de', 'df']
player 2's strategy: S2 = ['aa', 'ab', 'ba', 'bb']


In [7]:
# Example. Strategies in Matching Pennies Version B

X = list(range(7))
A = ['h', 't']
ADJ = [
    [None, 'h', 't', None, None, None, None],
    ['h', None, None, 'h', 't', None, None],
    ['t', None, None, None, None, 'h', 't',],
    [None, 'h', None, None, None, None, None],
    [None, 't', None, None, None, None, None],
    [None, None, 'h', None, None, None, None],
    [None, None, 't', None, None, None, None]
]
I = [0, 1]
H = [0, 1, 2, None, None, None, None]
iota = [0, 1, 1]

payoffs = {
    3: (-1, 1),
    4: (1, -1),
    5: (1, -1),
    6: (-1, 1)
}

mp_b = GameStructure(X, A, I, ADJ, H, iota, nature=False, payoffs=payoffs)
mp_b.display()
mp_b.display_payoff_matrix()

nodes: X = [0, 1, 2, 3, 4, 5, 6]
actions: A = ['h', 't']
players: I = [0, 1]
predecessors (X->X): P = [None, 0, 0, 1, 1, 2, 2]
successors: S = [[1, 2], [3, 4], [5, 6], None, None, None, None]
actions (X->A): alpha = [None, 'h', 't', 'h', 't', 'h', 't']
terminal nodes: T = [3, 4, 5, 6]
choices available at decision node: C(X) = [['h', 't'], ['h', 't'], ['h', 't']]
information sets (X->H): H = [0, 1, 2, None, None, None, None]
choices available at information set: C(H) = [['h', 't'], ['h', 't'], ['h', 't']]
players (H->I): iota = [0, 1, 1]
player 0's strategy: S0 = ['h', 't']
player 1's strategy: S1 = ['hh', 'ht', 'th', 'tt']

payoff matrix:
player 1 strategies:                                         
         hh       ht       th       tt   
player 0 h        (-1, 1)  (-1, 1)  (1, -1)  (1, -1) 
player 0 t        (1, -1)  (1, -1)  (-1, 1)  (-1, 1) 


In [8]:
# Example: Matching Pennies Version C

X = list(range(7))
A = ['h', 't']
ADJ = [
    [None, 'h', 't', None, None, None, None],
    ['h', None, None, 'h', 't', None, None],
    ['t', None, None, None, None, 'h', 't'],
    [None, 'h', None, None, None, None, None],
    [None, 't', None, None, None, None, None],
    [None, None, 'h', None, None, None, None],
    [None, None, 't', None, None, None, None]
]
I = [0, 1]
H = [0, 1, 1, None, None, None, None]
iota = [0, 1]

payoffs = {
    3: (-1, 1),
    4: (1, -1),
    5: (1, -1),
    6: (-1, 1)
}

mp_c = GameStructure(X, A, I, ADJ, H, iota, nature=False, payoffs=payoffs)
mp_c.display()
mp_c.display_payoff_matrix()

nodes: X = [0, 1, 2, 3, 4, 5, 6]
actions: A = ['h', 't']
players: I = [0, 1]
predecessors (X->X): P = [None, 0, 0, 1, 1, 2, 2]
successors: S = [[1, 2], [3, 4], [5, 6], None, None, None, None]
actions (X->A): alpha = [None, 'h', 't', 'h', 't', 'h', 't']
terminal nodes: T = [3, 4, 5, 6]
choices available at decision node: C(X) = [['h', 't'], ['h', 't'], ['h', 't']]
information sets (X->H): H = [0, 1, 1, None, None, None, None]
choices available at information set: C(H) = [['h', 't'], ['h', 't']]
players (H->I): iota = [0, 1]
player 0's strategy: S0 = ['h', 't']
player 1's strategy: S1 = ['h', 't']

payoff matrix:
player 1 strategies:                                         
         h        t    
player 0 h        (-1, 1)  (1, -1) 
player 0 t        (1, -1)  (-1, 1) 


In [9]:
# Example. Strategies in Matching Pennies Version D

X = list(range(15))
A = ['OT', 'FS', 'h', 't']
ADJ = [
    [None, 'OT', 'FS', None, None, None, None, None, None, None, None, None, None, None, None],
    ['OT', None, None, 'h', 't', None, None, None, None, None, None, None, None, None, None],
    ['FS', None, None, None, None, 'h', 't', None, None, None, None, None, None, None, None],
    [None, 'h', None, None, None, None, None, 'h', 't', None, None, None, None, None, None],
    [None, 't', None, None, None, None, None, None, None, 'h', 't', None, None, None, None],
    [None, None, 'h', None, None, None, None, None, None, None, None, 'h', 't', None, None],
    [None, None, 't', None, None, None, None, None, None, None, None, None, None, 'h', 't'],
    [None, None, None, 'h', None, None, None, None, None, None, None, None, None, None, None],
    [None, None, None, 't', None, None, None, None, None, None, None, None, None, None, None],
    [None, None, None, None, 'h', None, None, None, None, None, None, None, None, None, None],
    [None, None, None, None, 't', None, None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, 'h', None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, 't', None, None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, 'h', None, None, None, None, None, None, None, None],
    [None, None, None, None, None, None, 't', None, None, None, None, None, None, None, None]
]
I = [0, 1, 2] # player 0: Nature
H = [0, 1, 2, 3, 4, 5, 6, None, None, None, None, None, None, None, None]
iota = [0, 1, 2, 2, 2, 1, 1]
rho = [1/2, 1/2, None, None, None, None, None, None, None, None, None, None, None, None]

payoffs = {
    7: (-1, 1),
    8: (1, -1),
    9: (1, -1),
    10: (-1, 1),
    11: (-1, 1),
    12: (1, -1),
    13: (1, -1),
    14: (-1, 1),
}

mp_d = GameStructure(X, A, I, ADJ, H, iota, nature=True, rho=rho, payoffs=payoffs)
mp_d.display()
mp_d.display_expected_payoff_matrix()

nodes: X = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
actions: A = ['OT', 'FS', 'h', 't']
players: I = [0, 1, 2]
predecessors (X->X): P = [None, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6]
successors: S = [[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], None, None, None, None, None, None, None, None]
actions (X->A): alpha = [None, 'OT', 'FS', 'h', 't', 'h', 't', 'h', 't', 'h', 't', 'h', 't', 'h', 't']
terminal nodes: T = [7, 8, 9, 10, 11, 12, 13, 14]
choices available at decision node: C(X) = [['OT', 'FS'], ['h', 't'], ['h', 't'], ['h', 't'], ['h', 't'], ['h', 't'], ['h', 't']]
information sets (X->H): H = [0, 1, 2, 3, 4, 5, 6, None, None, None, None, None, None, None, None]
choices available at information set: C(H) = [['OT', 'FS'], ['h', 't'], ['h', 't'], ['h', 't'], ['h', 't'], ['h', 't'], ['h', 't']]
players (H->I): iota = [0, 1, 2, 2, 2, 1, 1]
player 0 is nature
probabilities(moves by nature) rho = [0.5, 0.5, None, None, None, None, None, None, None, None, None,